# 1. DATASET CREATION
*Creating Pandas Dataframe containing all the songs from [Songs used in commercials](https://www.songfacts.com/category/songs-used-in-commercials) and their Spotify ID.*

## 1.1. Importing Libraries

*Importing ``pandas`` and ``spotify.py`` module to access* Spotify *API.*

In [2]:
from spotify import SpotifyAPI
import pandas as pd

## 1.2. Reading Songs

*Reading songs from ``songs_from_songfacts.txt`` and creating a cleaner python list.*

In [ ]:
with open('songs_from_songfacts.txt') as f:
    data = f.readlines()
data = [line.replace('\n','').split(' - ') for line in data]

## 1.3. Accessing *Spotify* API

In [ ]:
client_id = '_CLIENT_ID_' # Add your client ID here
client_secret = '_SECRET_ID_' # Add your secret ID here
spoti = SpotifyAPI(client_id, client_secret)

## 1.4. Retrieving *Spotify* IDs

*Retrieving* Spotify *IDs for each song by searching title and artist.*

In [ ]:
for el in data:
    title = el[0]
    artist = el[1]
    res = spoti.search(title + ' ' + artist) # search func returns a result dict
    try:
        song = res['tracks']['items'][0]
        el.append(song['id'])
    except Exception:
        el.append('') 

## 1.5. DataFrame Creation and Exporting ``csv`` File

*Creating a ``pandas`` Dataframe.*

In [ ]:
songs_df = pd.DataFrame(data, columns = ['TITLE', 'ARTIST', 'ID'])

*Some wrong IDs found, replacing them after a manual check.*

In [ ]:
wrong_ids = [
        '6I0tz1wUfI3ibXHhjBdsv1',
        '3RTTs0HDk9rX2jqQa1BM1m',
        '7rgEfw0lhGuUM2Yrk950fJ',
        '50WEQ7sN1QT8xOEBSkLCgV',
        '4QFoJ4MirBaeDulxhifTEU',
        '5Tl3mhKpEAmso5QitemwJn',
        '5ATmWQHi5cAZgqBUzf8qS9',
        '57kR5SniQIbsbVoIjjOUDa'
        ]

right_ids = [
        '7bTlID6vzqECC5Vq61mysd',
        '5vVZaiK2mIL9WE1GWikOE6',
        '0H1GWNCOu7BcrxufcFgo9Z',
        '0DbcdVCzmY1IjilPTPaSOe',
        '2razPef7w7IoxNvG8plxsC',
        '0iC9wa4ARcVk0oV1zXpwjv',
        '3BfYDQC9SKBN1QCWVGb7C4',
        '7K6J5sDjzyON20FjMCW92Z'
        ]

songs_df['ID'] = songs_df['ID'].replace(wrong_ids, right_ids)

*Exporting the Dataframe as a ``csv`` file*

In [ ]:
songs_df.to_csv('songs.csv', index = False)